In [ ]:
!pip install spacy nltk pandas tqdm inflect
!python -m spacy download en_core_web_sm
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import os
import json
import random
import re
from pathlib import Path
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn
from tqdm import tqdm
nlp = spacy.load('en_core_web_sm')
import inflect
p = inflect.engine()

In [ ]:
DUTCH_COMPLEX_PATTERNS = {
    'premises': {
        'all': [
            {'tpl': 'Alle {a_pl} zijn {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Iedere {a_pl} behoort tot de groep van {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Zonder uitzondering zijn alle {a_pl} {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Alle {a_pl} vallen onder de categorie {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'De klasse van {a_pl} valt onder {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Indien iets {a_sg_art} is, dan is het tevens {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Elk individu dat {a_sg_art} is, kwalificeert zich als {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'De verzameling {a_pl} is een deelverzameling van {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Het is een vaststaand feit dat alle {a_pl} {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Elk exemplaar van {a_pl} is ook {b_sg_art}.', 'a_type': 'plural', 'b_type': 'singular_art'},
            {'tpl': 'Alles wat {a_sg_art} is, moet ook {b_sg_art} zijn.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Er bestaat geen {a_sg} die niet tevens {b_sg_art} is.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Wie {a_sg_art} is, is noodzakelijkerwijs {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'De totaliteit van de {a_pl} wordt gevormd door {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Alle {a_pl} maken deel uit van de categorie {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some': [
            {'tpl': 'Sommige {a_pl} zijn {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Er bestaan {a_pl} die {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Een deel van de {a_pl} is {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Enkele {a_pl} vertonen eigenschappen van {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Onder de {a_pl} zijn er sommige die {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Er is minstens één {a_sg} die {b_sg} is.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'In bepaalde gevallen is {a_sg_art} ook {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Men kan {a_pl} vinden die tot {b_pl} behoren.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Niet alle, maar zeker wel een paar {a_pl} zijn {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Er zijn voorbeelden van {a_pl} die als {b_pl} gelden.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Soms blijkt {a_sg_art} {b_sg_art} te zijn.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Een zeker percentage van de {a_pl} is {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Er zijn entiteiten die zowel {a_sg} als {b_sg} zijn.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Men treft {a_pl} aan die tevens {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Het komt voor dat {a_pl} ook {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Geen {a_pl} zijn {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Er zijn geen {a_pl} die {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': '{a_pl} en {b_pl} overlappen niet.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'De groep van {a_pl} bevat geen {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Absoluut geen {a_pl} behoren tot {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Geen enkele {a_sg} kan als {b_sg_art} worden beschouwd.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Niets wat {a_sg_art} is, is tegelijkertijd {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Het is uitgesloten dat {a_sg_art} {b_sg_art} is.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'De categorieën {a_pl} en {b_pl} zijn strikt gescheiden.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Onder geen beding is {a_sg_art} ook {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'De verzameling {a_pl} heeft geen raakvlakken met {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Als iets {a_sg_art} is, kan het onmogelijk {b_sg_art} zijn.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'In de gehele groep van {a_pl} tref je geen {b_sg} aan.', 'a_type': 'plural', 'b_type': 'singular'},
            {'tpl': 'Het is onjuist te beweren dat een {a_sg} {b_sg_art} is.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Totaal geen {a_pl} vallen binnen de klasse {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Sommige {a_pl} zijn geen {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Er zijn {a_pl} die niet {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Een deel van de {a_pl} behoort niet tot {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Niet alle {a_pl} zijn {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Onder de {a_pl} zijn er enkelen die geen {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Niet elke {a_sg} is {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Er is minstens één {a_sg} die niet {b_sg_art} is.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Bepaalde {a_pl} missen de eigenschap van {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Het is niet zo dat de totaliteit van de {a_pl} {b_pl} is.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Sommige exemplaren van {a_pl} vallen buiten {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Men kan {a_pl} aanwijzen die geen {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Een aanzienlijk deel van de {a_pl} is geen {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Soms blijkt een {a_sg} niet {b_sg_art} te zijn.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Niet alle leden van {a_pl} zijn tevens lid van {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Er zijn gevallen waarin {a_sg_art} geen {b_sg_art} is.', 'a_type': 'singular_art', 'b_type': 'singular_art'}
        ]
    },
    'conclusions': [
        'Dus, {conclusion}.',
        'Het volgt dat {conclusion}.',
        'Daaruit volgt dat {conclusion}.',
        'Hieruit concluderen we dat {conclusion}.',
        'Samenvattend: {conclusion}.',
        'Concluderend kunnen we stellen dat {conclusion}.',
        'Dit leidt tot de conclusie dat {conclusion}.',
        'Hierdoor is het duidelijk dat {conclusion}.',
        'Derhalve {conclusion}.',
        'Bijgevolg {conclusion}.'
    ]
}

In [ ]:
DUTCH_COMPLEX_PATTERNS_WITH_FOURTH_VAR = {
    'premises': {
        'all': [
            {'tpl': 'Elk {d_sg} dat {a_sg_art} is, is ook {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Als een {d_sg} {a_sg} is, dan is het {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Alle {d_pl} die {a_pl} zijn, zijn ook {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Zonder uitzondering geldt: elk {d_sg} met de eigenschap {a_sg} is {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Voor elk {d_sg} dat {a_sg_art} is, geldt dat het {b_sg_art} is.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Iedere {d_sg} die als {a_sg_art} kwalificeert, is noodzakelijkerwijs {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Wanneer {d_sg_art} tevens {a_sg_art} is, dan is het {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'De totaliteit van de {d_pl} die {a_pl} zijn, valt onder {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Binnen de groep {d_pl} zijn alle {a_pl} tevens {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Voor elke {d_sg} geldt: indien {a_sg_art}, dan {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Alles binnen {d_pl} wat {a_sg_art} is, behoort ook tot {b_pl}.', 'a_type': 'singular_art', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Geen enkele {d_sg} is {a_sg_art} zonder ook {b_sg_art} te zijn.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Alle leden van {d_pl} die tot {a_pl} behoren, zijn {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Wie in {d_pl} {a_sg_art} is, is altijd ook {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'plural'},
            {'tpl': 'Elk exemplaar van {d_pl} dat onder {a_pl} valt, is lid van {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'some': [
            {'tpl': 'Er bestaat ten minste één {d_sg} die zowel {a_sg} als {b_sg} is.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Sommige {d_pl} die {a_pl} zijn, zijn {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Een deel van de {d_pl} die {a_pl} zijn, behoort tot {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Er zijn {d_pl} die {a_pl} zijn en ook {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Ten minste één {d_sg} dat {a_sg_art} is, is {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Bepaalde {d_pl} vertonen zowel kenmerken van {a_pl} als van {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Het is mogelijk een {d_sg} te vinden die zowel {a_sg_art} als {b_sg_art} is.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Men treft {d_pl} aan die binnen de categorieën {a_pl} en {b_pl} vallen.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Soms is een lid van {d_pl} zowel {a_sg} als {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'plural'},
            {'tpl': 'In de groep {d_pl} bevinden zich exemplaren die {a_pl} en {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Er zijn situaties waarin {d_sg_art} tegelijk {a_sg_art} en {b_sg_art} is.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Onder de {d_pl} zijn er die de eigenschappen van {a_pl} en {b_pl} delen.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Men kan {d_pl} identificeren die zowel {a_pl} als {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Sommige elementen van {d_pl} zijn tevens {a_pl} en {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Er is sprake van {d_pl} die aan de criteria van {a_pl} en {b_pl} voldoen.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Geen van de {d_pl} die {a_pl} zijn, zijn {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Er bestaat geen {d_sg} dat zowel {a_sg} als {b_sg} is.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'De klasse van {d_pl} die {a_pl} zijn bevat geen {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Het is onmogelijk dat een {d_sg} zowel {a_sg} als {b_sg} is.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Geen enkel {d_sg} dat {a_sg} is, is {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Binnen {d_pl} sluit {a_pl} de eigenschap {b_pl} volledig uit.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Er zijn geen leden van {d_pl} die tevens {a_pl} en {b_pl} zijn.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Wie {d_sg_art} en {a_sg_art} is, kan nooit {b_sg_art} zijn.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'In de verzameling {d_pl} is geen enkel {a_sg_art} tevens {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'plural'},
            {'tpl': 'Het is onjuist te veronderstellen dat {d_sg_art} zowel {a_sg_art} als {b_sg_art} kan zijn.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Geen van de {d_pl} behoort tegelijk tot {a_pl} en {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Niets wat {d_sg_art} en {a_sg_art} is, is ook {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'De combinatie van {a_pl} en {b_pl} komt niet voor onder {d_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Geen {d_sg} verenigt de kenmerken van {a_sg} en {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Niemand in de groep {d_pl} is zowel {a_pl} als {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Niet elk {d_sg} dat {a_sg_art} is, is {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Er is minstens één {d_sg} dat {a_sg} is maar geen {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Sommige {d_pl} die {a_pl} zijn, zijn niet {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Een deel van de {d_pl} met eigenschap {a_pl} is uitgesloten van {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Er bestaan {d_pl} die {a_pl} zijn en toch geen {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Bepaalde leden van {d_pl} die {a_pl} zijn, missen de kwaliteit {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Het is niet het geval dat elke {d_sg} in {a_sg_art} ook {b_sg_art} is.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Men vindt {d_pl} die wel {a_pl} zijn, maar buiten {b_pl} vallen.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Minstens één {d_sg} bezit de eigenschap {a_sg}, maar niet {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Niet alle {d_pl} die {a_pl} zijn, worden als {b_pl} beschouwd.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Soms is {d_sg_art} wel {a_sg_art}, maar geen {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'In de groep {d_pl} zijn er {a_pl} die niet in {b_pl} passen.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Enkele exemplaren van {d_pl} zijn {a_pl} zonder {b_pl} te zijn.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Niet alles wat binnen {d_pl} als {a_sg} geldt, is tevens {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'plural'},
            {'tpl': 'Een zeker deel van de {d_pl} is {a_pl} maar behoort niet tot {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ]
    }
}

In [ ]:
def get_singular_and_plural(term):
    term = term.strip()
    sg_attempt = p.singular_noun(term)
    if sg_attempt is not False:
        sg = sg_attempt
        pl = term
    else:
        sg = term
        pl = p.plural_noun(term)
    if sg is False:
        sg = term
    if not pl:
        pl = term
    return sg, pl

In [ ]:
def get_article(term):
    term = term.strip()
    return p.an(term)

In [ ]:
def parse_sentence_for_type_and_terms(sentence):
    sentence = sentence.strip()
    # 4th-variable patterns first
    match = re.match(r'All (.*) that are (.*) are (.*)\\.', sentence)
    if match: return 'all', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are not (.*)\\.', sentence)
    if match: return 'some_not', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are (.*)\\.', sentence)
    if match: return 'some', match.group(1), match.group(2), match.group(3)
    match = re.match(r'No (.*) that are (.*) are (.*)\\.', sentence)
    if match: return 'no', match.group(1), match.group(2), match.group(3)
    # basic forms
    match = re.match(r'All (.*) are (.*)\\.', sentence)
    if match: return 'all', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are not (.*)\\.', sentence)
    if match: return 'some_not', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are (.*)\\.', sentence)
    if match: return 'some', None, match.group(1), match.group(2)
    match = re.match(r'No (.*) are (.*)\\.', sentence)
    if match: return 'no', None, match.group(1), match.group(2)
    raise ValueError(f'Could not parse sentence: {sentence}')

In [ ]:
def create_sub_dict(term_original_subject, term_original_predicate, term_original_fourth=None):
    sub_map = {}
    sg_a, pl_a = get_singular_and_plural(term_original_subject)
    sub_map['a_sg'] = sg_a
    sub_map['a_pl'] = pl_a
    sub_map['a_sg_art'] = get_article(sg_a)
    sg_b, pl_b = get_singular_and_plural(term_original_predicate)
    sub_map['b_sg'] = sg_b
    sub_map['b_pl'] = pl_b
    sub_map['b_sg_art'] = get_article(sg_b)
    if term_original_fourth:
        sg_d, pl_d = get_singular_and_plural(term_original_fourth)
        sub_map['d_sg'] = sg_d
        sub_map['d_pl'] = pl_d
        sub_map['d_sg_art'] = get_article(sg_d)
    return sub_map

In [ ]:
def safe_lowercase_first(s, sub_map):
    for val in sub_map.values():
        if s.startswith(val):
            return s
    return s[:1].lower() + s[1:]

In [ ]:
def convert_syllogism_to_complex(syllogism_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    simple_syllogism = syllogism_data['syllogism']
    # split sentences and keep trailing periods
    sentences = [s.strip() for s in simple_syllogism.split('.') if s.strip()]
    sentences = [s + '.' for s in sentences]
    premises_simple = sentences[:-1]
    conclusion_simple = sentences[-1]
    complex_premises = []
    for premise in premises_simple:
        p_type, p_fourth, p_subject, p_predicate = parse_sentence_for_type_and_terms(premise)
        sub_map = create_sub_dict(p_subject, p_predicate, p_fourth)
        if p_fourth:
            p_template = random.choice(complex_patterns_data_with_fourth_var['premises'][p_type])
        else:
            p_template = random.choice(complex_patterns_data['premises'][p_type])
        p_complex = p_template['tpl'].format(**sub_map)
        p_complex = p_complex[0].upper() + p_complex[1:]
        complex_premises.append(p_complex)
    c_type, c_fourth, c_subject, c_predicate = parse_sentence_for_type_and_terms(conclusion_simple)
    c_sub_map = create_sub_dict(c_subject, c_predicate, c_fourth)
    if c_fourth:
        c_premise_template = random.choice(complex_patterns_data_with_fourth_var['premises'][c_type])
    else:
        c_premise_template = random.choice(complex_patterns_data['premises'][c_type])
    c_premise_complex = c_premise_template['tpl'].format(**c_sub_map)
    c_premise_complex = safe_lowercase_first(c_premise_complex, c_sub_map)
    c_wrapper_template = random.choice(complex_patterns_data['conclusions'])
    c_complex = c_wrapper_template.format(conclusion=c_premise_complex)
    c_complex = c_complex[:-1] if c_complex.endswith('.') else c_complex
    c_complex = c_complex[0].upper() + c_complex[1:]
    final_complex_syllogism = ' '.join(complex_premises) + ' ' + c_complex
    new_data = syllogism_data.copy()
    new_data['syllogism'] = final_complex_syllogism
    return new_data

In [ ]:
def process_dataset(input_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    complex_dataset = []
    for item in input_data:
        try:
            complex_item = convert_syllogism_to_complex(item, complex_patterns_data, complex_patterns_data_with_fourth_var)
            complex_dataset.append(complex_item)
        except ValueError as e:
            print(f" Skipping item ID {item.get('id', 'N/A')} due to parsing error: {e}")
            continue
    return complex_dataset

In [ ]:
# Example data and run (sanity check)
sample_data = [
    {'id': '1', 'syllogism': 'All cats are animals. Some pets are cats. Some pets are animals.', 'validity': True},
    {'id': '2', 'syllogism': 'No cars are birds. Some vehicles are cars. Some vehicles are not birds.', 'validity': True}
]
complex_syllogism_data = process_dataset(sample_data, DUTCH_COMPLEX_PATTERNS, DUTCH_COMPLEX_PATTERNS_WITH_FOURTH_VAR)
print('\n--- Converted Dutch Complex Syllogisms ---')
print(json.dumps(complex_syllogism_data, indent=4, ensure_ascii=False))